In [37]:
# Import necessary modules
import os
import tempfile
import subprocess
from plyfile import PlyData
from PIL import Image
import IPython.display as display


In [39]:
# Define paths for input and output
ply_file_path = r"D:\sunny\Codes\DPS\data_teethseg\origin\000101_origin.ply"
output_uv_layout_dir = r"D:\sunny\Codes\DPS\data_bpy_map"

# Load PLY file using plyfile
ply_data = PlyData.read(ply_file_path)

# Convert PLY data to a format that Blender can import
vertices = [(vertex['x'], vertex['y'], vertex['z']) for vertex in ply_data['vertex']]
faces = [list(face[0]) for face in ply_data['face']]

In [ ]:
blender_script = """
import bpy
import os

# Define a log function
def log(message):
    with open("{log_file}", "a") as log:
        log.write(message + "\\n")
    print(message)

log("Creating new mesh")
# Create a new mesh
mesh = bpy.data.meshes.new(name="PLYMesh")
obj = bpy.data.objects.new(name="PLYObject", object_data=mesh)
bpy.context.collection.objects.link(obj)
bpy.context.view_layer.objects.active = obj
obj.select_set(True)

log("Creating mesh from vertices and faces")
# Create mesh from given vertices and faces
mesh.from_pydata({vertices}, [], {faces})
mesh.update()

log("Entering edit mode")
# Ensure we are in edit mode
bpy.ops.object.mode_set(mode='EDIT')

log("Selecting the entire mesh")
# Select the entire mesh
bpy.ops.mesh.select_all(action='SELECT')

log("Performing Smart UV Project")
# Perform Smart UV Project to mark seams
bpy.ops.uv.smart_project(angle_limit=66.0, island_margin=0.02)

log("Switching to object mode")
# Switch to object mode to access UV layout
bpy.ops.object.mode_set(mode='OBJECT')

log("Exporting UV layout to PNG")
# Save the UV layout as a PNG file
output_uv_layout_path = os.path.join("{output_uv_layout_dir}", "uv_layout.png")
bpy.ops.uv.export_layout(filepath=output_uv_layout_path, size=(1024, 1024), export_all=True)

log("Quitting Blender")
# Close Blender after execution
bpy.ops.wm.quit_blender()
"""

In [61]:
# Format the script with the vertices, faces, and output path
blender_script = blender_script.format(
    vertices=vertices, 
    faces=faces,
    output_uv_layout_dir=output_uv_layout_dir
)

# Save the script to a temporary file using tempfile
with tempfile.NamedTemporaryFile(delete=False, suffix=".py") as temp_file:
    temp_file.write(blender_script.encode('utf-8'))
    script_filepath = temp_file.name

# Path to the Blender executable
blender_executable = r"D:\Program Files\Blender Foundation\Blender 4.2\blender.exe"


In [62]:
# Check if the Blender executable exists
if not os.path.exists(blender_executable):
    raise FileNotFoundError(f"Blender executable not found at {blender_executable}")

# Check if the script file exists
if not os.path.exists(script_filepath):
    raise FileNotFoundError(f"Script file not found at {script_filepath}")

In [63]:
# Run Blender in background mode with the script and capture the output
result = subprocess.run(
    [blender_executable, "--background", "--python", script_filepath],
    capture_output=True, text=True
)


# Print Blender's output for debugging
print(result.stdout)
print(result.stderr)

# Check for errors in Blender's execution
if result.returncode != 0:
    raise RuntimeError(f"Blender execution failed: {result.stderr}")


Blender 4.2.0 (hash a51f293548ad built 2024-07-16 06:29:33)

Blender quit

Traceback (most recent call last):
  File "C:\Users\sunny\AppData\Local\Temp\tmpj16k_exp.py", line 35, in <module>
    offscreen = gpu.types.GPUOffScreen(width, height)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
SystemError: GPU functions for drawing are not available in background mode



In [65]:
# Run Blender interactively with the script
subprocess.run([blender_executable, "--python", script_filepath])

# Load the generated UV layout image
uv_layout_image_path = os.path.join(output_uv_layout_dir, "uv_layout.png")
if not os.path.exists(uv_layout_image_path):
    raise FileNotFoundError(f"UV layout image not found at {uv_layout_image_path}")

uv_layout_image = Image.open(uv_layout_image_path)

# Display the UV layout image
display.display(uv_layout_image)

FileNotFoundError: UV layout image not found at D:\sunny\Codes\DPS\data_bpy_map\uv_layout.png